In [2]:
import pandas as pd
import dask.dataframe as dd
from dask_ml.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from tqdm import tqdm
from sklearn.tree import DecisionTreeClassifier
import pickle

In [3]:
X_train = dd.read_parquet('X_train.parquet', engine='fastparquet')
X_test = dd.read_parquet('X_test.parquet', engine='fastparquet')
y_train = dd.read_parquet('y_train.parquet', engine='fastparquet')

messages = pd.read_csv('messages.csv')
test_intervals = pd.read_csv('test_intervals.csv')

In [4]:
display(y_train.head())

,Y_ЭКСГАУСТЕР А/М №9_ЗАПОРНАЯ АРМАТУРА ЭКСГАУСТЕРА №9,Y_ЭКСГАУСТЕР А/М №9_МАСЛОСТАНЦИЯ ЖИДКОЙ СМАЗКИ ЭКСГ. №9,Y_ЭКСГАУСТЕР А/М №9_ЭЛЕКТРОАППАРАТУРА ЭКСГ. №9,Y_ЭКСГАУСТЕР А/М №9_КЛ1 ТР№9 ДО ЭД ЭКСГАУСТЕРА №9,Y_ЭКСГАУСТЕР А/М №9_ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4 ЭКСГ. №9,Y_ЭКСГАУСТЕР А/М №9_КОРПУС ЭКСГ. №9,Y_ЭКСГАУСТЕР А/М №9_РЕДУКТОР ГАЗ. ЗАДВИЖКИ ЭКСГ. №9,Y_ЭКСГАУСТЕР А/М №9_МАСЛООХЛАДИТЕЛЬ М-05-1 ЭКСГ. №9,Y_ЭКСГАУСТЕР А/М №9_ЭЛ/ДВИГАТЕЛЬ ГАЗ. ЗАДВИЖКИ ЭКСГ. №9,Y_ЭКСГАУСТЕР А/М №9_ЭЛЕКТРООБОРУДОВАНИЯ ЭКСГАУСТЕРА №9,...,Y_ЭКСГАУСТЕР А/М №6_ЭКСГАУСТЕР А/М №6,Y_ЭКСГАУСТЕР А/М №6_ПОДШИПНИК ОПОРНЫЙ ЭКСГ. №6,Y_ЭКСГАУСТЕР А/М №6_ТИРИСТОРНЫЙ ВОЗБУДИТЕЛЬ ТВ-400 ЭКСГ6 ВУ1,Y_ЭКСГАУСТЕР А/М №6_ДВИГАТЕЛЬ ПУСКОВОГО МАСЛОНАСОСА ЭКСГ. №6,Y_ЭКСГАУСТЕР А/М №6_МАСЛОНАСОС ШЕСТЕРЕНЧАТЫЙ (ПУСК.) ЭКСГ.№6,Y_ЭКСГАУСТЕР А/М №6_ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№6 Т.3,Y_ЭКСГАУСТЕР А/М №6_МАСЛОПРОВОДЫ ЭКСГ №6,Y_ЭКСГАУСТЕР А/М №6_КЛ1 ТР№6 ДО ЭД ЭКСГАУСТЕРА №6,Y_ЭКСГАУСТЕР А/М №6_КЛ2 ТР№6 ДО ЭД ЭКСГАУСТЕРА №6,Y_ЭКСГАУСТЕР А/М №6_ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№6 Т.2
DT,,,,,,,,,,,,,,,,,,,,,
2019-01-16 13:21:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-16 13:21:10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-16 13:21:20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-16 13:21:30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-16 13:21:40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
def concat(x, y, number=None):
    number = number
    feature_prefix = 'ЭКСГАУСТЕР '
    target_prefix = 'Y_ЭКСГАУСТЕР А/М №'
    
    try:
        train_columns = [column for column in x.columns if feature_prefix + str(number) in column]
        target_columns = [column for column in y.columns if target_prefix + str(number) in column]
        train_frame = x[train_columns]
        target_frame = y[target_columns]
        
    except ValueError:
        print("Oops! You broke my function...")
        
    return train_frame, target_frame

In [5]:
five_first = concat(X_train, y_train, 5)

In [8]:
train_frame, target_frame  = concat(X_train, y_train, 5)

features = train_frame
features = features.fillna(0)
target = target_frame.iloc[:, 4]
target = target.astype(int)

features_train, features_test, target_train, target_test = train_test_split(
                                                            features, 
                                                            target,
                                                            random_state=666,
                                                            test_size=0.25,
                                                            shuffle=True)


model = DecisionTreeClassifier()
model.fit(features_train, target_train)
pred = model.predict(features_test)

print("F-1 score for machine 5, detail 4:", f1_score(target_test, pred, average='macro'))

F-1 score for machine 5, detail 4: 0.9935883886773076


In [8]:
machine_numbers = [4, 6, 7, 8, 9]

for number in machine_numbers:
    train_frame, target_frame = concat(X_train, y_train, number)
    features = train_frame
    features = features.fillna(0)
    print(number)
    for feature in range(len(target_frame.columns)):
        target = target_frame.iloc[:, feature]
        target = target.astype(int)
        model = DecisionTreeClassifier()
        model.fit(features, target)
        test_features = X_test[features.columns].fillna(0)
        predict = model.predict(test_features)
        predictions = pd.DataFrame(predict, index=X_test.index)
        file_name =  str(number) + '_' + str(feature)
        predictions.to_csv(file_name)
        decision_tree_pkl_filename = 'decision_tree_classifier_' + file_name + '.pkl'
        decision_tree_model_pkl = open(decision_tree_pkl_filename, 'wb')
        pickle.dump(model, decision_tree_model_pkl)
        decision_tree_model_pkl.close()

        print(feature)

6
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
7
0
1


KeyboardInterrupt: 

In [5]:
machine_numbers = [4, 6, 7, 8, 9]

def concat_predictons():
    all_targets = pd.DataFrame(index=X_test.index)
    for number in machine_numbers:
        train_frame, target_frame = concat(X_train, y_train, number)
        features = train_frame
        features = features.fillna(0)
        for feature in range(len(target_frame.columns)):
            current_target = pd.read_csv('predictions/' + str(number) + '_' + str(feature))
            all_targets = pd.concat([all_targets, current_target])
    all_targets.columns = y_train.columns
    file_name = 'predictions.csv'
    all_targets.to_csv(file_name)

In [6]:
concat_predictons()

NameError: name 'X_test' is not defined

In [6]:
def get_machine_targets(number):
    target_prefix = 'Y_ЭКСГАУСТЕР А/М №'
    target_columns = [column for column in y_train.columns if target_prefix + str(number) in column]
    return target_columns

In [9]:
def set_columns_names():
    for number in machine_numbers:
        target_names = get_machine_targets(number)
        for i in range(len(target_names)):
            target = pd.read_csv('predictions/' + str(number) + '_' + str(i))
            target.columns = ['DT', target_names[i]]
            file_name =  'pred33/' + str(number) + '_' + str(i)
            target.to_csv(file_name)

In [ ]:
def machine_intervals(data):
    intervals_detail = pd.DataFrame()
    data = data[:10000]
    m1_intervals = []
    m3_intervals = []

    m1 = False
    m3 = False

    start_time = 0
    finish_time = 0

    for index, row in data.iterrows():
        if row['error_type'] != 0:
            if m1 == False and m3 == False:
                start_time = row['DT']
                if row['error_type'] == 1:
                    m1 = True
                else: 
                    m3 = True
                if (row['error_type'] == 1) and m1 == True and m3 == False:
                    finish_time = row['DT']
                    interval = [start_time, finish_time]
                    m1_intervals.append(interval)
                    m1 = False
                    m3 = True

                if (row['error_type'] == 2) and m1 == False and m3 == True:
                    finish_time = row['DT']
                    interval = [start_time, finish_time]
                    m3_intervals.append(interval)
                    m3 = False
                    m1 = True

        else:
            finish_time = row['DT']
            interval = [start_time, finish_time]
            if m1 == True:
                m1_intervals.append(interval)
                m1 = False
            if m3 == True:
                m3_intervals.append(interval)
                m3 = False
                
    return m1_intervals, m3_intervals